In [ ]:
#default_exp parse

# Parsing a roll call pdf
> Extracting voting information from roll call vote PDF files (those with RCV in their file name), which were downloaded via `eu_parliament.download`.

In [ ]:
#hide
%load_ext autoreload
%autoreload 2

**Some useful references:**

- PyPDF2 docs: https://pythonhosted.org/PyPDF2/PdfFileReader.html
    - `PageObject`: https://pythonhosted.org/PyPDF2/PageObject.html#PyPDF2.pdf.PageObject

**Content of RCV pdfs:**
- intro
- table of contents
- votes by issue (split by outcome +/-/0, i.e. yes/no/abstain and party):
    - every MEP is represented by the surname
    - each issue may also have vote corrections (only MEP and no party info and for some reasonthe MEPs's names here include their forename) 

**TODO:** 
- Some MEP names seem to disappear when reading the PDF, like Kolakušić, and some are incorrectly read like Ždanoka $\rightarrow$ ”danoka $\rightarrow$ fix in `SummaryParser` / `VotesParser` or PyPDF2
    - consequence: the total vote count and the counted names of the actually parsed MEPs disagrees
    - tried `pdfminer`, which initially looked promising (it's not dropping words as far as I can see) but actually copies and pasts bits of texts all over the place

In [ ]:
#export
import PyPDF2 as pdf
from pathlib import Path
import typing
import re
import pandas as pd
import collections
import plotly.express as px
import plotly.graph_objects as go
import os
from eu_parliament import download
from pdfminer import high_level

Listing available PDFs

In [ ]:
#export
def extract_date_from_filename(file): return pd.to_datetime(re.search(r'(\d{4}-\d{2}-\d{2})', file).group())

def list_available_pdfs(file_dir:typing.Union[str, Path]=download.PDF_PATH, 
                        filter_fun:typing.Callable=lambda x: 'RCV' in x):
    
    pdf_files = [Path(file_dir)/file for file in os.listdir(file_dir) if filter_fun(file)]
    file_timestamps = [extract_date_from_filename(file.name) for file in pdf_files]
    
    return pd.DataFrame({'file': pdf_files, 'timestamp': file_timestamps})

In [ ]:
rcv_pdfs = list_available_pdfs(); rcv_pdfs.head()

In [ ]:
path = rcv_pdfs.iloc[0,0]; path

In [ ]:
#export
class Pdf2Text:
    'Parsing a PDF to a list of strings'
    
    def __call__(self, path:typing.Union[Path,str], 
                 method:str='pdfminer', **kwargs):
        
        fun = getattr(self, f'parse_with_{method}')
        return fun(path, **kwargs)
    
    @staticmethod
    def parse_with_pypdf2(path:typing.Union[Path,str]):
        
        with open(path, 'rb') as f:
            reader = pdf.PdfFileReader(f)

            all_texts = [reader.getPage(i).extractText()  
                         for i in range(reader.getNumPages())]
        return all_texts
    
    @staticmethod
    def parse_with_pdfminer(path:typing.Union[Path,str],
                            codec:str='UTF-8'):
        
        with open(path, 'rb') as f:
            string = high_level.extract_text(f, codec=codec)
            
        all_texts = re.split(r'(PE\s\d{3}\.\d{3})', string)
        all_texts = [s0+s1 for s0,s1 in zip(all_texts[::2], all_texts[1::2])]
        return all_texts

Using `pdfminer` to parse the PDF

In [ ]:
%%time
all_texts = Pdf2Text()(path, method='pdfminer')

In [ ]:
#hide
assert isinstance(all_texts, list)
assert all([isinstance(text, str) for text in all_texts])

Using `PyPDF2` to parse the PDF

In [ ]:
%%time
all_texts2 = Pdf2Text()(path, method='pypdf2')

In [ ]:
#hide
assert isinstance(all_texts2, list)
assert all([isinstance(text, str) for text in all_texts2])

In [ ]:
#hide
assert len(all_texts) == len(all_texts2)

In [ ]:
print(all_texts[0])

In [ ]:
print(all_texts2[0])

Comparing the extracted texts we see that PyPDF2 seems to drop some words whereas pdfminer does not seem to do so.

## Parsing the summary

Defining two utility functions to make our lifes easier

In [ ]:
#export
def get_ix(strings:typing.List[str], pattern_fun:typing.Callable): # , must_match:bool=True
    return [i for i, s in enumerate(strings) if pattern_fun(s)] # (must_match and s==pattern) or (not must_match and pattern in s)

def useful_string(s):  return ('docx' not in s) and (len(s)>0) \
            and (s != ' ')

In [ ]:
#export
class SummaryParser:
    'Parsing the summary of the issues voted on and documented in an RCV file'
    
    #pattern = re.compile('(\.\.+)')
    is_toc = lambda self, text: 'SOMMAIRE' in text or 'CONTENTS' in text
    
    def __init__(self, all_texts:typing.List[str],
                 method:str):
        assert method in ['pdfminer', 'pypdf2']
        
        self.method = method
        self.num_pages = len(all_texts)
        
        self.ix_start, summary_text = self.get_start(all_texts)

        self.vote_names, self.vote_page_numbers = self.parse_names_and_page_numbers(summary_text)
        
        self.ix_end, summary_text = self.get_end(all_texts, summary_text)
        
        if self.ix_end > self.ix_start:
            self.vote_names, self.vote_page_numbers = self.parse_names_and_page_numbers(summary_text)
        
    def get_start(self, all_texts:typing.List[str]): 
        ix_start = get_ix(all_texts, self.is_toc)[0]        
        return ix_start, all_texts[ix_start]
        
    def pypdf2_parse_names_and_page_numbers(self, all_texts:typing.List[str]): 
        texts = re.split(r'(\d+\.\n?)(?=\s*[a-zA-Z])', all_texts)
        
        ix_content = get_ix(texts, self.is_toc)[0]
        texts = texts[ix_content+1:]

        given_number = [text.strip() for text in texts[::2]]

        contents = [text.strip() for text in texts[1::2]]
        page_numbers, vote_names = [], []
        
        for text in contents:
            if 'docx' in text:
                _text = text.split('\n')
                _ix = get_ix(_text, lambda x: 'docx' in x)[0]
                _text = '\n'.join(_text[:_ix])
            else:
                _text = text
            try:
                re.search(r'(\d+)$', _text).group()
            except:
                print('failed at parsing', [text], 'to', [_text])
            page_numbers.append(int(re.search(r'(\d+)$', _text).group()))
            vote_names.append(re.sub(r'\.*\d+$', '', _text).strip())
        
        assert len(vote_names) == len(page_numbers)
        assert len(set(page_numbers)) == len(page_numbers), collections.Counter(page_numbers).most_common()
        
        return vote_names, [nr-1 for nr in page_numbers]
    
    def pdfminer_parse_names_and_page_numbers(self, all_texts:typing.List[str]): 
        texts = re.split(r'(?<=\d)\n\n', all_texts)
        #print(texts)
        
        texts = [text for text in texts if 'docx' not in text and re.search(r'(\d{3}\.\d{3})', text) is None]
        #some strings do not properly begin with a number, rather it is somewhere in the middle
        ix_messed_up = [i for i, text in enumerate(texts) if text[0].isalpha()]
        #print(ix_messed_up)
        print([texts[i] for i in ix_messed_up])
        misplaced_numbers = [re.search(r'(\n\n\d\.\n)', texts[i]).group() for i in ix_messed_up]
        for i, number in zip(ix_messed_up, misplaced_numbers):
            texts[i] = re.sub(r'(\n\n\d\.\n)', ' ', texts[i])
            texts[i] = number.lstrip() + texts[i]
        print(texts)
        
        ix_content = get_ix(texts, self.is_toc)[0]
        texts = texts[ix_content+1:]

        given_number = [text.strip() for text in texts[::2]]

        contents = [text.strip() for text in texts[1::2]]
        #[text0.strip()+text1.strip() for text0, text1 in zip(texts[1::3],
        #                                                                texts[2::3])]
        #print(given_number)
        #print(contents)
        page_numbers, vote_names = [], []
        
        for text in contents:
            if 'docx' in text:
                print('\ntööööööxt', text)
                _text = text.split('\n')
                _ix = get_ix(_text, lambda x: 'docx' in x)[0]
                _text = '\n'.join(_text[:_ix])
            else:
                _text = text
            try:
                re.search(r'(\d+)$', _text).group()
            except:
                print('failed at parsing', [text], 'to', [_text])
            print(_text)    
            page_numbers.append(int(re.search(r'(\d+)$', _text).group()))
            vote_names.append(re.sub(r'\.*\d+$', '', _text).strip())
        
        assert len(vote_names) == len(page_numbers)
        assert len(set(page_numbers)) == len(page_numbers), collections.Counter(page_numbers).most_common()
        
        return vote_names, [nr-1 for nr in page_numbers]
        
    def parse_names_and_page_numbers(self, all_texts:typing.List[str]): 
        
        return getattr(self, f'{self.method}_parse_names_and_page_numbers')(all_texts)
        
    def get_end(self, all_texts:typing.List[str], summary_text:str): 
        
        if self.ix_start + 1 == self.vote_page_numbers[0]:
            return self.ix_start, summary_text
        
        ix_end = self.vote_page_numbers[0]
        summary_text = '\n'.join(all_texts[self.ix_start:ix_end])
        
        return ix_end, summary_text
    
    @property
    def df(self):
        return pd.DataFrame({
            'vote name': self.vote_names,
            'start page': self.vote_page_numbers,
            'end page': [nr - 1 for nr in self.vote_page_numbers[1:]] + [self.num_pages]
        })
    
    

In [ ]:
all_texts[2]

In [ ]:
all_texts2[2]

In [ ]:
summary = SummaryParser(all_texts, method='pdfminer')
df_summary = summary.df
df_summary.head(150)

In [ ]:
summary = SummaryParser(all_texts2, method='pypdf2')
df_summary = summary.df
df_summary.head(150)

In [ ]:
with pd.option_context('display.max_rows', 150):
    display(df_summary)

## Parsing the votes

In [ ]:
#export
class VotesParser:
    
    def __init__(self, start_page:int, end_page:int, all_texts:typing.List[str]):
        '`end_page` is inclusive'
        assert start_page <= end_page
        self.start_page = start_page
        self.end_page = end_page
        
        votes_texts = self.preprocess(all_texts)
        
        original_votes, vote_corrections = self.check_for_corrections(votes_texts)
        
        #print(original_votes, vote_corrections)
        
        self.vote_counts, self.votes = self.parse_votes(original_votes)
        
        if vote_corrections is not None:
            self.corrections = self.parse_vote_corrections(vote_corrections)
    
    @property
    def df_votes(self):
        df = []
        for outcome in self.votes:
            for party in self.votes[outcome]:
                df.extend([(mep, outcome, party) for mep in self.votes[outcome][party]])
        return pd.DataFrame(df, columns=['MEP', 'vote', 'Party'])
    
    @property
    def df_corrections(self):
        df = []
        for outcome in self.votes:
            df.extend([(mep, outcome) for mep in self.corrections[outcome]])
        return pd.DataFrame(df, columns=['MEP', 'vote'])
    
    def preprocess(self, all_texts:typing.List[str]):
        votes_texts = "\n".join(all_texts[self.start_page:self.end_page+1])
        
        s = slice(self.start_page, self.end_page+1)
        votes_texts = "\n".join(all_texts[s]).split("\n")
        
        votes_texts = [text for text in votes_texts if \
                       useful_string(text)]
        return votes_texts
    
    def parse_votes(self, votes_texts:typing.List[str]):
        #print(votes_texts)
        
        ix_yes, ix_no, ix_abstain = self.get_yes_no_abstain_indices(votes_texts)
        
        counts = {
            'yes': int(votes_texts[ix_yes-1]),
            'no': int(votes_texts[ix_no-1]),
            'abstain': int(votes_texts[ix_abstain-1]),
        }
        votes = {
            'yes': self.get_mep_and_party_vote_for_outcome(votes_texts[ix_yes:ix_no-1]),
            'no': self.get_mep_and_party_vote_for_outcome(votes_texts[ix_no:ix_abstain-1]),
            'abstain': self.get_mep_and_party_vote_for_outcome(votes_texts[ix_abstain:])
        }
        #print(counts)
        #print([(k, (x:=[len(val) for val in v.values()]), sum(x)) for k,v in votes.items()])
        return counts, votes
    
    def get_yes_no_abstain_indices(self, votes_texts):
        ix_yes = min(get_ix(votes_texts, lambda x: x == '+'))
        ix_no = get_ix(votes_texts, lambda x: x == '-')[0]
        ix_abstain = max(get_ix(votes_texts, lambda x: x == '0'))
        assert ix_yes > 0
        assert ix_yes < ix_no
        assert ix_no < ix_abstain
        return ix_yes, ix_no, ix_abstain
    
    def parse_vote_corrections(self, votes_corrections_texts:typing.List[str]): 

        ix_yes, ix_no, ix_abstain = self.get_yes_no_abstain_indices(votes_corrections_texts)
        
        yes_meps = '\n'.join(votes_corrections_texts[ix_yes+1:ix_no]).split(',')
        no_meps = '\n'.join(votes_corrections_texts[ix_no+1:ix_abstain]).split(',')
        abstain_meps = '\n'.join(votes_corrections_texts[ix_abstain+1:]).split(',')
        
        return {
            'yes': [self.process_mep(mep) for mep in yes_meps if useful_string(mep.strip())],
            'no': [self.process_mep(mep) for mep in no_meps if useful_string(mep.strip())],
            'abstain': [self.process_mep(mep) for mep in abstain_meps if useful_string(mep.strip())]
        }
        
    def get_mep_and_party_vote_for_outcome(self, votes_texts:typing.List[str]):

        ix_party = [(i,text) for i, text in enumerate(votes_texts)\
                    if len(text)>0 and (':' == text[0] or ':' == text[-1])]
        
        ix_party = [i if (text[-1]==':' and len(text)>1) else i-1 for i, text in ix_party]
        parties = [votes_texts[ix] for ix in ix_party]
        meps = [','.join(votes_texts[ix0+1:ix1]).split(',') for ix0, ix1 in 
                zip(ix_party, ix_party[1:]+[len(votes_texts)])]
        votes = {party: [self.process_mep(mep) for mep in _meps if useful_string(mep.strip())] for party, _meps in zip(parties, meps)}
        
        return votes
    
    def process_mep(self, mep:str, placeholder:str='unknown mep'):
        mep = mep.strip()
        if ':' in mep:
            if mep.startswith(':'):
                mep = mep[1:]
            if mep.endswith(':'):
                mep = mep[:-1]
        if mep == '':
            mep = placeholder
        return mep
        
    def check_for_corrections(self, votes_texts:typing.List[str]): 
        
        ix = get_ix(votes_texts, lambda x: 'CORRECCIONES E INTENCIONES DE VOTO' in x)
        assert len(ix) <= 1
        no_corrections = len(ix) == 0
        if no_corrections:
            return votes_texts, None
        else:
            return votes_texts[:ix[0]], votes_texts[ix[0]:]

In [ ]:
%%time
start, end = summary.df.iloc[0,1:]
votes = VotesParser(start, end, all_texts)
display(votes.df_votes, votes.df_corrections)

In [ ]:
df_summary = summary.df
df_summary.head()

Parsing all issues voted on

In [ ]:
#export
def get_all_issues(df_summary:pd.DataFrame, all_texts:typing.List[str]):
    all_votes, all_corrections = [], []
    all_counts = []

    for i,row in df_summary.iterrows():
        votes = VotesParser(row['start page'], row['end page'], all_texts)

        df_votes = votes.df_votes
        df_votes['vote name'] = row['vote name']
        all_votes.append(df_votes)

        counts = votes.vote_counts
        counts['vote name'] = row['vote name']
        all_counts.append(counts)

        df_corrections = votes.df_corrections
        df_corrections['vote name'] = row['vote name']
        all_corrections.append(df_corrections)

    all_counts = pd.DataFrame(all_counts)
    all_votes = pd.concat(all_votes)
    all_corrections = pd.concat(all_corrections)
    return all_counts, all_votes, all_corrections

In [ ]:
%%time
all_counts, all_votes, all_corrections = get_all_issues(df_summary, all_texts)

In [ ]:
all_counts.head(), all_counts.tail()

In [ ]:
all_votes.head(), all_votes.tail()

Overlap between vote names

In [ ]:
s1 = set(all_counts['vote name'].unique())
s2 = set(all_votes['vote name'].unique())

assert len(s1.intersection(s2))/len(s1.union(s2)) == 1.

## Visualisation of total counts vs identified counts

In [ ]:
all_votes_summed = all_counts.copy()
outcomes = ['yes', 'no', 'abstain']
all_votes_summed.loc[:,outcomes] = None

for outcome in outcomes:
    vote_mask = all_votes['vote'] == outcome
    
    for vote in all_votes['vote name'].unique():
        mask = all_votes_summed['vote name'] == vote
        _votes = (all_votes['vote name'] == vote) & vote_mask
        all_votes_summed.loc[mask, outcome] = _votes.sum()
    
    all_votes_summed[outcome] = all_votes_summed[outcome].astype(int)
    all_votes_summed[f'{outcome}-delta'] = all_votes_summed[outcome] - all_counts[outcome]
        
all_votes_summed

In [ ]:
all_votes_summed.info()

In [ ]:
pd.melt(all_votes_summed, value_vars=['yes','no','abstain'])

In [ ]:
deltas = pd.melt(all_votes_summed, value_vars=[v for v in all_votes_summed.columns if 'delta' in v])
deltas

In [ ]:
display(deltas.groupby('variable').describe())
px.histogram(data_frame=deltas,
             x=f'value',
             color='variable',
             barmode='overlay', nbins=21,
             title=f'Discrepancy for \'{outcome}\' votes')